<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#循环神经网络-(recurrent-neural-network,-RNN)" data-toc-modified-id="循环神经网络-(recurrent-neural-network,-RNN)-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>循环神经网络 (recurrent neural network, RNN)</a></span></li><li><span><a href="#长短时记忆网络-(LTSM)" data-toc-modified-id="长短时记忆网络-(LTSM)-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>长短时记忆网络 (LTSM)</a></span></li><li><span><a href="#循环神经网络的变种" data-toc-modified-id="循环神经网络的变种-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>循环神经网络的变种</a></span><ul class="toc-item"><li><span><a href="#双向循环神经网络-(bidirectional-RNN)" data-toc-modified-id="双向循环神经网络-(bidirectional-RNN)-3.1"><span class="toc-item-num">3.1&nbsp;&nbsp;</span>双向循环神经网络 (bidirectional RNN)</a></span></li><li><span><a href="#深层循环神经网络-(deepRNN)" data-toc-modified-id="深层循环神经网络-(deepRNN)-3.2"><span class="toc-item-num">3.2&nbsp;&nbsp;</span>深层循环神经网络 (deepRNN)</a></span></li><li><span><a href="#循环神经网络的dropout" data-toc-modified-id="循环神经网络的dropout-3.3"><span class="toc-item-num">3.3&nbsp;&nbsp;</span>循环神经网络的dropout</a></span></li></ul></li></ul></div>

## 循环神经网络 (recurrent neural network, RNN)
[Link1](https://www.imooc.com/article/23821)


> 循环神经网络的主要用途是处理和预测`序列数据`。循环神经网络的隐藏层之间的结点是有连接的，隐藏层的输入不仅包括输入层的输出，还包括上一时刻隐藏层的输出。

> 循环神经网络会对于每一个时刻的输入结合当前模型的状态给出一个输出。循环神经网络在每一个时刻会有一个输入X(t)，然后根据循环神经网络当前的状态A(t)提供一个输出H(t)；而循环神经网络当前的状态A(t)是根据上一时刻的状态A(t-1)和当前的输入X(t)共同决定的。

> 对于一个序列数据，可以将这个序列上不同时刻的数据依次传入循环神经网络的输入层，而输出可以是对序列中下一个时刻的预测，也可以是对当前时刻信息的处理结果。

> 在循环神经网络中，循环体网络结构中的参数`在不同时刻是共享的`。

> 循环神经网络的总损失为所有时刻(或者部分时刻)上的损失函数的总和。

![RNN前向传播计算示意图](illustration/RNN前向传播计算示意图.png)

## 长短时记忆网络 (LTSM)

> 循环神经网络工作的关键点是使用历史的信息来帮助当前的决策；但同时也带来了更大的技术挑战 -- 长期依赖(long-term dependencies)

> LSTM的提出就是为了解决长期依赖问题。LSTM是一种拥有三个"门"结构的特殊网络结构。(所谓"门"的结构就是一个使用sigmoid神经网络和和一个按位做乘法的操作，这两个操作合在一起就是一个"门"结构，之所以该结构叫做"门"是因为使用sigmoid作为激活函数的全连接神经网络层会输出一个0到1之间的数值，描述当前输入有多少信息量可以通过这个结构)

> "遗忘门"会根据当前的输入`x(t)`, 上一时刻状态`c(t-1)`和上一时刻输出`h(t-1)`共同决定哪一部分记忆需要被遗忘

> "输入门"会根据`x(t)`、`c(t-1)`和`h(t-1)`决定哪些部分将进入当前时刻的状态`c(t)`

> "输出门"会根据最新的状态`c(t)`、上一时刻的输出`h(t-1)`和当前的输入`x(t)`来决定该时刻的输出`h(t)`


![LSTM单元结构示意图](illustration/LSTM单元结构示意图.png)


In [ ]:
# 定义一个LSTM结构, 并指定隐藏层的大小
lstm = tf.contrib.rnn.BasicLSTMCell(lstm_hidden_size)
'''
__init__(
    num_units,
    forget_bias=1.0,
    state_is_tuple=True,
    activation=None,
    reuse=None,
    name=None,
    dtype=None
)
'''

# 将LSTM中的状态初始化为全0数组. 和其他神经网络类似, 在优化神经网络时, 
# 每次也会使用一个batch的训练样本, 以下代码中, batch_size给出了一个batch大小
# BasicLSTMCell类提供了zero_state函数来生成全0的初始状态
state = lstm.zero_state(batch_size, tf.float32)
'''
zero_state(batch_size, dtype)
Args:
    batch_size:  int, float, or unit Tensor representing the batch size.
    dtype:  the data type to use for the state.
Return:
    zero-filled state tensor(s).    
'''

# 虽然理论上循环神经网络可以处理任意长度的序列, 但是在训练时为了避免梯度消散的问题,
# 通常会规定一个最大的序列长度, 以下代码中, 用 num_steps 来表示这个长度
for i in range(num_steps):
    # 在第一个时刻声明LSTM结构中使用的变量
    # 在之后的时刻都需要复用之前定义好的变量
    if i > 0:
        tf.get_variable_scope().reuse_variables()
    # 每一步处理时间序列中的一个时刻。将当前输入(current_input)和前一时刻状态(state)
    # 传入定义的LSTM结构可以得到当前LSTM结构的输出lstm_output和更新后的状态state
    lstm_output, state = lstm(current_input, state)
    # 将当前时刻LSTM结构的输出传入一个全连接层得到最后的输出
    final_output = fully_connected(lstm_output)
    

## 循环神经网络的变种

### 双向循环神经网络 (bidirectional RNN)

双向循环神经网络是由两个循环神经网络上下叠加在一起组成的。输出由这两个循环神经网络的状态共同决定。  
![双向循环神经网络结构示意图](illustration/双向循环神经网络结构示意图.png)  

> 双向循环神经网络的主体结构就是两个单向循环神经网络的结合。在每一个时刻`t`, 输入会同时提供给这两个方向相反的循环神经网络, 而输出则是由这两个单向循环神经网络共同决定。双向循环神经网络的前向传播过程和单向的循环神经网络十分类似

### 深层循环神经网络 (deepRNN)
深层循环神经网络在每一个时刻上将循环体结构复制了多次，每一层的循环体中参数是一致的，而不同层中的参数可以不同   
![深层循环神经网络结构示意图](illustration/深层循环神经网络结构示意图.png)

### 循环神经网络的dropout
在循环神经网络中使用`dropout`可以使网络更加健壮；区别于卷积神经网络只在最后的全连接层中使用`dropout`, 循环神经网络一般只在不同层循环体结构之间使用`dropout`, 而不在同一层的循环体结构之间使用。也就是说从时刻`t-1`传递到时刻`t`时，循环神经网络不会进行状态的`dropout`；而在同一时刻`t`中，不同层循环体之间会使用`dropout`  
![深层循环神经网络使用dropout示意图](illustration/深层循环神经网络使用dropout示意图.png)

> 图中实线箭头表示不使用`dropout`，虚线箭头表示使用`dropout`